<a href="https://colab.research.google.com/github/akiseakusa/car_projection/blob/main/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First to remove noise in provided mask, I have use erosion followed by dilution

Then algin the wall and floor using image stiching and superimpose the car on the stitched image

car shadow not added,because of following reason

*   shadow_mask.size not eqaul to car_image.size
*   Do not know the direction of source(we can generate the shadow)
*   Tried to resize and add according to car mask end,but shadow can be of either side of car, otherwise it may mask other region too if add extra margin in space






In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image,ImageOps

In [ ]:
# to mitigate noise in mask

# Load the image (assuming it's a binary mask)
image = cv2.imread('/content/4.png', 0)  # Load as grayscale

# Apply a binary threshold to ensure it's a binary image
_, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

# Define a kernel for morphological operations
kernel = np.ones((5, 5), np.uint8)  # Adjust the size of the kernel if needed

# Use morphological opening to remove small noise (erosion followed by dilation)
clean_image = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel)

# Optionally, use median blur to further smooth the mask and remove small dots
clean_image_blur = cv2.medianBlur(clean_image, 5)

# Display the results
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(binary_image, cmap='gray')

plt.subplot(1, 2, 2)
plt.title("Cleaned Image")
plt.imshow(clean_image_blur, cmap='gray')

plt.show()

# Save the cleaned image
cv2.imwrite('/content/mask/4.png', clean_image_blur)


In [ ]:
#----------------correct code add scaling for floor and wall-------------------
# Load images
car_image = Image.open('/content/images/2.jpeg')
mask_image = Image.open('/content/mask/2.png').convert('L')  # Convert mask to grayscale
floor_image = Image.open('/content/floor.png')
wall_image = Image.open('/content/wall.png')

# Resize the floor and wall images to fit the car image
wall_width, wall_height = wall_image.size
floor_width, floor_height = floor_image.size

new_height = wall_height + floor_height
# Create a new background image
background = Image.new("RGB", (wall_width, new_height))

# Paste the wall at the top and floor at the bottom
background.paste(wall_image, (0, 0))  # Paste wall at the top
background.paste(floor_image, (0, new_height - floor_height))  # Paste floor at the bottom
background_resized = background.resize(mask_image.size)

# Get the bounding box of the car from the mask
car_bbox = mask_image.getbbox()  # (left, upper, right, lower)

# Vertical shift value (positive for downward, negative for upward)
vertical_shift = 200  # Example: move the car down by 50 pixels

# Shift the bounding box of the car to apply the vertical shift
left, upper, right, lower = car_bbox
upper_shifted = upper + vertical_shift
lower_shifted = lower + vertical_shift

# Define a new bounding box based on the vertical shift
car_bbox_shifted = (left, upper_shifted, right, lower_shifted)

# Now shift the mask image by creating a new mask with the shifted bounding box
shifted_mask = Image.new("L", mask_image.size, 0)
shifted_mask.paste(mask_image, (0, vertical_shift))  # Apply the vertical shift to the mask

# Create a new blank image to paste the car image in the shifted position
shifted_car_image = Image.new("RGBA", car_image.size)
shifted_car_image.paste(car_image, (0, vertical_shift))  # Apply the same shift to the car image

# Apply the shifted mask to combine the car with the new background
composite_image = Image.composite(shifted_car_image, background_resized, shifted_mask)

# Save the final image
output_path = '/content/output_image_shifted.png'
composite_image.save(output_path)

# Convert the car image to a format compatible with matplotlib
car_image_np = np.array(composite_image)

# Create a figure and axis to display the final image
plt.figure(figsize=(8, 6))
plt.imshow(car_image_np)
plt.axis('off')  # Turn off axis numbers and ticks

# # Draw a red rectangle around the car's shifted bounding box
# rectangle = plt.Rectangle((left, upper_shifted), right - left, lower_shifted - upper_shifted,
#                           edgecolor='red', facecolor='none', linewidth=2)
# plt.gca().add_patch(rectangle)

# Show the result
plt.show()

# Output path
print("Image saved to:", output_path)


In [ ]:
#correct do not change


# Load images
car_image = Image.open('/content/images/2.jpeg')
mask_image = Image.open('/content/mask/2.png').convert('L')  # Convert mask to grayscale
floor_image = Image.open('/content/floor.png')
wall_image = Image.open('/content/wall.png')
shadow_image = Image.open('shadow_mask/2.png')
car_width, car_height = car_image.size
padded_shadow_mask = ImageOps.pad(shadow_image, (car_width, car_height), color=(0, 0, 0))
padded_shadow_mask = padded_shadow_mask.convert("L")
# Resize the floor and wall images to fit the car image
wall_width, wall_height = wall_image.size
floor_width, floor_height = floor_image.size

new_height = wall_height+floor_height
# Create a new background image
background = Image.new("RGB", (wall_width, new_height))

# Paste the wall at the top and floor at the bottom
background.paste(wall_image, (0, 0))  # Paste wall at the top
background.paste(floor_image, (0, new_height- floor_height))  # Paste floor at the bottom
background_resized = background.resize(mask_image.size)
# Apply the mask to combine the car with the new background
pre_composite_image = Image.composite(car_image, background_resized, mask_image)
composite_image = Image.composite(car_image, pre_composite_image , padded_shadow_mask)
# Save the final image
output_path = '/content/output_image.png'
composite_image.save(output_path)
car_bbox = mask_image.getbbox()  # (left, upper, right, lower)

# Convert the car image to a format compatible with matplotlib
car_image_np = np.array(composite_image)

# Create a figure and axis
plt.figure(figsize=(8, 6))
plt.imshow(car_image_np)
plt.axis('off')  # Turn off axis numbers and ticks

left, upper, right, lower = car_bbox
rectangle = plt.Rectangle((left, upper), right - left, lower - upper,
                          edgecolor='red', facecolor='none', linewidth=2)
plt.gca().add_patch(rectangle)
# Output path
print("Image saved to:", output_path)
